In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'] , axis=1)

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
df.Attrition_Flag.value_counts().plot(kind='bar')
plt.title('Attrition Flag', '')
plt.show()

Transform other categorical variables

In [ ]:
for var in ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']: 
    df[var].value_counts().plot(kind='bar')
    plt.title(var)
    plt.show()

# Preprocessing categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

labels = ['Attrition_Flag', 'Gender', 'Income_Category']
categories = ['Education_Level', 'Marital_Status', 'Card_Category']

le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)

In [ ]:
df[labels]=df[labels].apply(lambda x: le.fit_transform(x))

In [ ]:
for cat in categories:
    cat_features= pd.DataFrame(ohe.fit_transform(df[[cat]]), columns=list([df[cat].unique()]))
    df = pd.concat([df, cat_features], axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df1=df.iloc[:, :24]
df2=df.iloc[:, 25:30]
df3=df.iloc[:, 31:]
df_final = pd.concat([df1, df2, df3], axis=1)

In [ ]:
df_final.columns

In [ ]:
X = df_final.drop(columns=categories).drop(columns=['Attrition_Flag']).drop(columns=['CLIENTNUM'])
y = df_final['Attrition_Flag']
X.info()
y.shape

# Split data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=31)

In [ ]:
#Define scoring metrics: Precision on class 0
from sklearn.metrics import recall_score, accuracy_score, make_scorer
custom_recall = make_scorer(recall_score, pos_label=0)

### Build model

In [ ]:
#Build XGB model
from xgboost import XGBClassifier


In [ ]:
Test_score = []
Train_score = []
n_estimators = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for n_est in n_estimators:
    xgb = XGBClassifier(objective='binary:logistic', learning_rate=0.1, n_estimators=n_est)
    xgb.fit(X_train, y_train)
    y_train_pred = xgb.predict(X_train)
    Train_score.append(recall_score(y_train, y_train_pred, pos_label=0))
    y_test_pred = xgb.predict(X_test)
    Test_score.append(recall_score(y_test, y_test_pred, pos_label=0))

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(Train_score)
ax.plot(Test_score, c = 'red')
ax.set_xticklabels(n_estimators)
ax.set_ylim(0.8, 1.1)
plt.show()

In [ ]:
Test_score = []
Train_score = []
learning_rate = list(np.linspace(0.05, 0.25, 10))

for lr in learning_rate:
    xgb = XGBClassifier(objective='binary:logistic', learning_rate=lr, n_estimators=400)
    xgb.fit(X_train, y_train)
    y_train_pred = xgb.predict(X_train)
    Train_score.append(recall_score(y_train, y_train_pred, pos_label=0))
    y_test_pred = xgb.predict(X_test)
    Test_score.append(recall_score(y_test, y_test_pred, pos_label=0))
    
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(Train_score)
ax.plot(Test_score, c = 'red')
ax.set_xticklabels(learning_rate)
ax.set_ylim(0.8, 1.1)
plt.show()

In [ ]:
xgb = XGBClassifier(objective='binary:logistic', learning_rate=0.1225, n_estimators=400)

# Parameters tuning

In [ ]:
#Build ParamsGrid
params = {
    'max_depth': [2, 3, 4],
    'subsample': [0.5, 0.65, 0.8],
    'colsample_bytree': [0.1, 0.4, 0.7],
    'lambda' : [0.3, 0.5, 0.7, 0.9]
}

In [ ]:
#RandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

clf = RandomizedSearchCV(estimator=xgb, param_distributions=params, cv=5, scoring=custom_recall, return_train_score = True)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.cv_results_

# Evaluation

In [ ]:
y_test_pred = clf.best_estimator_.predict(X_test)
recall_score(y_test, y_test_pred, pos_label=0)

In [ ]:
accuracy_score(y_test, y_test_pred)